# Example Notebook

## 1. Vertex and Edge
The classes `Vertex` and `Edge` provides the principal objects of the class `Graph`.
The package `Graph` provides a class of the same name. 

In [1]:
from vertex import Vertex

The class vertex has a `name`, a `weight`, a `contain` which can be any object and an uptader.

In [2]:
v1=Vertex("v1",-1,contain="Isabella",updaters=[])

The attribute `contain` refers to *any kind* object we want to store on the vertex, in this case we put the string `"Isabella"`. But it can be a list

In [3]:
v1=Vertex("v1",-1,contain=["Isabella","Fernanda","Lorena"],updaters=[])

Or can be a dictonary

In [4]:
v1=Vertex("v1",-1,contain={"Isabella":24,"Fernanda":27,"Lorena":22},updaters=[])

The attribute *updaters* is a dictionary, the keys is a string identifyer and the value is a double.

In [5]:
v1=Vertex("v1",-1,contain={"Isabella":24,"Fernanda":27,"Lorena":22},updaters=["debt", "remaining_days"])

You can recover the value of the updater by using the function

In [6]:
v1.get_updater_val("debt")

0

All the updaters are initialized at zero, but you cand modify it:

In [7]:
v1.update_val("debt",2500)
v1.get_updater_val("debt")

2500

There are more methods on this class, like `add_updater()`, `delete_updater()`, and getters and setters.

In [8]:
print(v1)
Vertex?

Vertex (name='v1',weight=-1)


The edges connects the vertexes. It has a `start` vertex, an `end` vertex, `cost`, `name` and `updaters`.

In [9]:
from edge import Edge

In [10]:
v1=Vertex("v1",25,contain="Isabella",updaters=[])
v2=Vertex("v2",25,contain="Fernanda",updaters=[])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])

It also have the usual getters and setters and works pretty similar to a vertex.

In [11]:
print(e1)
Edge?

Edge (v1,v2)


## 2. Graph

This class models the network given by the vertexes and edges. It has a dict of vertexes (`vertex`), a dict of edges (`edges`), a `name`, can be directed (`isdirected: True | False`), can be autoupdated (`autoupdate: True | False`) when you add information, has a dict of `updaters` and finally can be `propagative`.

In [12]:
from graph import Graph
Graph?

In [13]:
v1=Vertex("v1",25,contain="Isabella",updaters=[])
v2=Vertex("v2",25,contain="Fernanda",updaters=[])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])

g1=Graph(vertex=[v1,v2],edges=[e1])
print(g1)

Graph (
 vertex=[Vertex (name='v1',weight=25), Vertex (name='v2',weight=25)],
 edges=[Edge (v1,v2)])


In [14]:
g1.get_vertexes()

[Vertex (name='v1',weight=25), Vertex (name='v2',weight=25)]

There is no need to specify the vertexes, you cas just pass the edges:

In [15]:
v1=Vertex("v1",25,contain="Isabella",updaters=[])
v2=Vertex("v2",25,contain="Fernanda",updaters=[])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])

g1=Graph(edges=[e1])
print(g1)

Graph (
 vertex=[Vertex (name='v1',weight=25), Vertex (name='v2',weight=25)],
 edges=[Edge (v1,v2)])


### 2.1 Uptaters: What is it?

The idea of the updaters is to change the value of some registers on the graph following an undater rule. So, for example, you want to save the sum of the weights entering to your vertex from the neighborhoods multiplied by the cost of the edge connecting them, also when you put a new vertex on the graph you would like to update this information. Then you define a new updater caller `"in_weight"`, also, creates a rule that defines the way you update this information. So you create a function which take a vertex sum weight of the values of the neighborhoods and save it on the updater of the vertex.

In [16]:
def in_weight(graph,key,start_vertex,vertex=None,edge=None,back=False):
    for v in graph.get_edges(vertex,starting=False):
        vertex.update_val(key,vertex.get_updater_val(key)+v.get_cost()*v.start.get_weight())

In [17]:
v1=Vertex("v1",25,contain="Isabella",updaters=["in_weight"])
v2=Vertex("v2",25,contain="Fernanda",updaters=["in_weight"])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])

g1=Graph(edges=[e1],updaters={"in_weight":in_weight})

**Before update**

In [18]:
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0, Vertex (name='v2',weight=25): 0}

In [19]:
g1.update_all()

**After update**

In [20]:
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0, Vertex (name='v2',weight=25): 62.5}

If we add a new vertex and an edge we can recalculate the value of the updater

In [21]:
v3=Vertex("v3",10,contain="Lorena",updaters=["in_weight"])

e2=Edge(v3,v2,cost=1,name="Rent",updaters=[])

g1.add_edge([e2])

In [22]:
g1.update_all()
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0,
 Vertex (name='v2',weight=25): 135.0,
 Vertex (name='v3',weight=10): 0}

You can notice the new graph looks like

                  (weight=25)v1 ------------->v2<------------- v3(weight=10)
                                  (cost=2.5)        (cost=1)

So the value of the updater would be `25*2.5+10*1=72.5`, but the last cell shows the value `135.0`. Thats because out function `in_weight` takes the *actual* value of the updater and sums the value of the weight times the cost. To fix this, redefine the function 

In [23]:
def in_weight(graph,key,start_vertex,vertex=None,edge=None,back=False):
    vertex.update_val(key,0) # Reset the updater
    for v in graph.get_edges(vertex,starting=False):
        vertex.update_val(key,vertex.get_updater_val(key)+v.get_cost()*v.start.get_weight())

In [24]:
v1=Vertex("v1",25,contain="Isabella",updaters=["in_weight"])
v2=Vertex("v2",25,contain="Fernanda",updaters=["in_weight"])
v3=Vertex("v3",10,contain="Lorena",updaters=["in_weight"])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])
e2=Edge(v3,v2,cost=1,name="Rent",updaters=[])

g1=Graph(edges=[e1,e2],updaters={"in_weight":in_weight})
g1.update_all()
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0,
 Vertex (name='v2',weight=25): 72.5,
 Vertex (name='v3',weight=10): 0}

So the rule that updates the follows your function.

#### Autoupdate

In the last graph we includes a new vertex and a new edge. Also we used the function `update_all()` to recalculate all the values with the new information. If we need this action to be automatically when a new edge is added we cand set `autoupdate=True`.

In [25]:
v1=Vertex("v1",25,contain="Isabella",updaters=["in_weight"])
v2=Vertex("v2",25,contain="Fernanda",updaters=["in_weight"])

e1=Edge(v1,v2,cost=2.5,name="Debt",updaters=[])

g1=Graph(edges=[e1],autoupdate=True,updaters={"in_weight":in_weight})
g1.update_all()
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0, Vertex (name='v2',weight=25): 62.5}

In [26]:
# We add a new edge
v3=Vertex("v3",10,contain="Lorena",updaters=["in_weight"])

e2=Edge(v3,v2,cost=1,name="Rent",updaters=[])

g1.add_edge([e2])

In [27]:
# New values
g1.get_updaters_values("in_weight")

{Vertex (name='v1',weight=25): 0,
 Vertex (name='v2',weight=25): 72.5,
 Vertex (name='v3',weight=10): 0}

#### Propagation

The `autoupdate` function has a particularity: it updates the start vertes and the end vertex of the added edge. So, for example, we store the data `carry` on the some start vertexes `v1` and `v2`, on the graph

In [28]:
v1=Vertex("v1",25,contain="Isabella",updaters=["carry"])
v2=Vertex("v2",25,contain="Fernanda",updaters=["carry"])
v3=Vertex("v3",25,contain="Fernanda",updaters=["carry"])
v4=Vertex("v4",25,contain="Fernanda",updaters=["carry"])

e1=Edge(v1,v3,cost=1,name="net",updaters=[])
e3=Edge(v3,v4,cost=1,name="net",updaters=[])

g2=Graph(edges=[e1,e3],autoupdate=True)

So, we need to define the updater function to the updater `carry`

In [29]:
def carry(graph,key,start_vertex,vertex=None,edge=None,back=False):
    vertex.update_val(key,0) # Reset the updater
    for v in graph.get_edges(vertex,starting=False):
        vertex.update_val(key,vertex.get_updater_val(key)+v.get_cost()*v.start.get_updater_val(key))
g2.add_func_updater("carry",carry)

Now we set new values for the updater at vertex `v1`

In [30]:
g2.set_vertex_updater_val(v1,"carry",2)
g2.get_updaters_values("carry")

{Vertex (name='v1',weight=25): 2,
 Vertex (name='v3',weight=25): 2,
 Vertex (name='v4',weight=25): 0}

As you may notice, the update value was updated for the vertex `v3`, which is conected to the vertex `v1`, but the vertex `v4` must be `2` too, because the value of `v3` changed. If we set `propagation=True` the graph will continue updating all the vertex conected to the new value.

In [31]:
v1=Vertex("v1",25,contain="Isabella",updaters=["carry"])
v2=Vertex("v2",25,contain="Fernanda",updaters=["carry"])
v3=Vertex("v3",25,contain="Fernanda",updaters=["carry"])
v4=Vertex("v4",25,contain="Fernanda",updaters=["carry"])
v5=Vertex("v5",25,contain="Fernanda",updaters=["carry"])

e1=Edge(v1,v3,cost=1,name="net",updaters=[])
e3=Edge(v3,v4,cost=1,name="net",updaters=[])
e4=Edge(v4,v5,cost=1,name="net",updaters=[])
g2=Graph(edges=[e1,e3,e4],autoupdate=True,propagation=True)

Your graph have propagation, you need to specify the start vertex


In [32]:
g2.get_updaters_values("carry")

{Vertex (name='v1',weight=25): 0,
 Vertex (name='v3',weight=25): 0,
 Vertex (name='v4',weight=25): 0,
 Vertex (name='v5',weight=25): 0}

The las message was given by the propagation function. Due to the graph has `autoupdate=True` it updates all the graph when constructed, by propagation it tries to propagate values, but the graph don't know where to start.

In [33]:
g2.set_vertex_updater_val(v1,"carry",2)
g2.get_updaters_values("carry")

{Vertex (name='v1',weight=25): 2,
 Vertex (name='v3',weight=25): 2,
 Vertex (name='v4',weight=25): 2,
 Vertex (name='v5',weight=25): 2}

Now, we the vertex `v4` have the correct value.

If we add a new vertex and edge, the graph also propagate.

In [34]:
e2=Edge(v2,v3,cost=1,name="net",updaters=[])
g2.add_edge([e2])
g2.get_updaters_values("carry")

{Vertex (name='v1',weight=25): 2,
 Vertex (name='v3',weight=25): 2,
 Vertex (name='v4',weight=25): 2,
 Vertex (name='v5',weight=25): 2,
 Vertex (name='v2',weight=25): 0}

In [35]:
g2.set_vertex_updater_val(v2,"carry",2)
g2.get_updaters_values("carry")

{Vertex (name='v1',weight=25): 2,
 Vertex (name='v3',weight=25): 4,
 Vertex (name='v4',weight=25): 4,
 Vertex (name='v5',weight=25): 4,
 Vertex (name='v2',weight=25): 2}

In [36]:
def p_back(graph,key,start_vertex,vertex=None,edge=None,back=False):
    if back:
        for v in graph.get_edges(vertex,starting=False):
            v.start.update_val(key,v.start.get_updater_val(key)+v.get_cost()*vertex.get_updater_val(key))
    else:
        for v in graph.get_edges(vertex,ending=False):
            v.end.update_val(key,v.end.get_updater_val(key)+v.get_cost()*vertex.get_updater_val(key)+vertex.get_bias())

## 3. Full connected neural net

An special kind of graph is the full connected neural net (`NeuralNetFC`)

In [37]:
from neural_net_FC import NeuralNetFC

This class have a `name`, a list with the neurons per layer (`npl`), a list with `updaters`, and it initializes the edges with random values by default (`random=True | False`)

In [38]:
nn=NeuralNetFC(name="example",npl=[2,3,4],updaters={"pr":p_back},random=True)

Connecting network: 100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


In [39]:
nn

NeuralNet_FC (
    config =[2, 3, 4]
    vertex =['L0_v0', 'L0_v1', 'L1_v0', 'L1_v1', 'L1_v2', 'L2_v0', 'L2_v1', 'L2_v2', 'L2_v3'],
    edges =[Edge (L0_v0,L1_v0), Edge (L0_v0,L1_v1), Edge (L0_v0,L1_v2), Edge (L0_v1,L1_v0), Edge (L0_v1,L1_v1), Edge (L0_v1,L1_v2), Edge (L1_v0,L2_v0), Edge (L1_v0,L2_v1), Edge (L1_v0,L2_v2), Edge (L1_v0,L2_v3), Edge (L1_v1,L2_v0), Edge (L1_v1,L2_v1), Edge (L1_v1,L2_v2), Edge (L1_v1,L2_v3), Edge (L1_v2,L2_v0), Edge (L1_v2,L2_v1), Edge (L1_v2,L2_v2), Edge (L1_v2,L2_v3)])

The class creates all the neurons and the edges. The name of the neurons follows the rule `Li_vj`, where `i` is the number of the layer and `j` is the number of the neuron on the layer. This class loads the class `Neuron`, a subclass of vertex.

In [40]:
nn.find_vertex_by_name("L0_v0")

Neuron (name='L0_v0, bias=0)

In [41]:
nn.predict([[1,1]],key=["pr"])

Prop. on 3 layers: 100%|█████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]


[[1.5752352287753222,
  1.815585087664564,
  1.0618740597253393,
  0.7136011862078434]]

### 3.1 Draw a Fully connected Neural Net

The class `DrawGraph` provides a visualization of the neural net using OpenGL and Pyglet.

In [42]:
from draw_graph import DrawGraph
DrawGraph(nn,x_sep=150,y_sep=100).run()

In [43]:
nn2=NeuralNetFC(name="example",npl=[4,2,1,5,3],updaters={"pr":p_back},random=True)
DrawGraph(nn2,x_sep=150,y_sep=100).run()

Connecting network: 100%|████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]


***
#### Disclaimer

This document and the files are work on progress.